In [3]:
import sys
from collections import Counter, defaultdict

from PYEVALB import scorer
from PYEVALB import parser, tree
from nltk.tree import *

In [4]:
pred_seqs_file = '/home/ta/Projects/RNNG_all/rnng_self/data/post_processing/pred_seqs.txt'
test_seqs_file = '/home/ta/Projects/RNNG_all/rnng_self/data/post_processing/test_seqs.txt'
train_seqs_file = '/home/ta/Projects/RNNG_all/rnng_self/data/post_processing/train_seqs.txt'
dev_seqs_file = '/home/ta/Projects/RNNG_all/rnng_self/data/post_processing/dev_seqs.txt'
test_seqs = open(test_seqs_file, 'r').readlines()
pred_seqs = open(pred_seqs_file, 'r').readlines()

In [5]:
import re
def no_X_grammar(grammar):
    no_X_grammar = []
    for rule in grammar:
        # print (rule)
        if str(rule.lhs()) == 'XX':
            # print ('XX -> sth = ', rule)
            continue
        rule = str(rule).replace('XX', '')
        rule = re.sub(' +', ' ', rule)
        rule = rule.strip()
        no_X_grammar.append(rule)
    return set(no_X_grammar)


def get_grammar(seq_file, no_XX):
    def no_X_rule(rule):
        if str(rule.lhs()) == 'XX':
            # print ('XX -> sth = ', rule)
            return None
        # rule = str(rule).replace('XX', '')
        rule = re.sub('XX', '', str(rule))
        rule = re.sub(' +', ' ', rule)
        rule = rule.strip()
        return rule

    grammar = set()
    f = open(seq_file, 'r')
    seqs = f.readlines()
    print ('Get grammar: len of', seq_file, '=', len(seqs))
#     print ('Getting grammar...')
    rule_counter = Counter()
    for line in seqs:
        tree = Tree.fromstring(line)
        rules = tree.productions()
        grammar.update(rules)
        if no_XX:
            no_X_rules = [no_X_rule(rule) for rule in tree.productions()]
            no_X_rules = [rule for rule in no_X_rules if rule is not None]
            rules = no_X_rules
        rule_counter.update(rules)

    if no_XX:
        assert len(no_X_grammar(grammar)) == len(rule_counter)
        return no_X_grammar(grammar), rule_counter
    else:
        return grammar, rule_counter

In [17]:
train_grammar, _ = get_grammar(train_seqs_file, no_XX=False) 
dev_grammar, _ = get_grammar(dev_seqs_file, no_XX=False)
train_grammar.update(dev_grammar) 
# print (train_grammar[:5])

Get grammar: len of /home/ta/Projects/RNNG_all/rnng_self/data/post_processing/train_seqs.txt = 39831
Get grammar: len of /home/ta/Projects/RNNG_all/rnng_self/data/post_processing/dev_seqs.txt = 1700


In [19]:
print (list(train_grammar)[:5])
print (len(train_grammar))  
example = (list(train_grammar)[0]) 
print (str(example))

[VP -> XX PRT S S XX S, XX -> 'refers', XX -> 'noble', XX -> 'downgrade', XX -> 'anxiously']
29284
VP -> XX PRT S S XX S


In [8]:
def get_grammar_from_file_new(seq_file):
    def no_quote_prod(prod):
        prod = re.sub('\'', '', str(prod))
        prod = re.sub(' +', ' ', prod)
        return prod.strip()

    print ('Getting grammar from', seq_file)
    f = open(seq_file, 'r')
    # grammar = None 
    prod_counter = Counter() 
    prods = []
    cnt_line = 0
    for seq in f:
        cnt_line += 1
        tree = parser.create_from_bracket_string(seq)
        this_seq_prods, _ = tree.productions(skip_XX=False, skip_span=True)
        this_seq_prods = [no_quote_prod(prod) for prod in this_seq_prods]
        prods.extend(this_seq_prods) 
        prod_counter.update(this_seq_prods)

    print ('Done at', cnt_line, 'lines.')
    print ('There are', len(set(prods)), 'productions')
    print ('Grammar:', prod_counter.most_common(10))
    print ('')
    return set(prods), prod_counter

train_grammar_self, _ = get_grammar_from_file_new(train_seqs_file)
train_grammar_self.update(get_grammar_from_file_new(dev_seqs_file)[0])


Getting grammar from /home/ta/Projects/RNNG_all/rnng_self/data/post_processing/train_seqs.txt
Done at 39831 lines.
There are 29162 productions
Grammar: [('NP -> XX XX', 90469), ('PP -> XX NP', 85682), ('NP -> XX', 65595), ('XX -> ,', 48723), ('XX -> the', 41107), ('NP -> XX XX XX', 40655), ('XX -> .', 39019), ('TOP -> S', 36165), ('VP -> XX VP', 35836), ('NP -> NP PP', 34965)]

Getting grammar from /home/ta/Projects/RNNG_all/rnng_self/data/post_processing/dev_seqs.txt
Done at 1700 lines.
There are 6554 productions
Grammar: [('NP -> XX XX', 3777), ('PP -> XX NP', 3519), ('NP -> XX', 2543), ('XX -> ,', 2143), ('XX -> the', 1873), ('NP -> XX XX XX', 1729), ('XX -> .', 1660), ('TOP -> S', 1521), ('NP -> NP PP', 1452), ('VP -> XX VP', 1345)]



In [23]:
train_grammar_self = list(train_grammar_self)
print (list(train_grammar_self)[:5])


['XX -> inch', 'TOP -> SBARQ', 'XX -> fleet', 'XX -> USG', 'XX -> own']
29283


In [27]:
train_grammar_str = [str(e) for e in train_grammar] 
print (len(train_grammar_self), len(train_grammar_str))
print (train_grammar_str[0]) 
print (train_grammar_self[0])
outside = [e for e in train_grammar_self if e not in train_grammar_str] 
outside = [e for e in outside if 'XX ->' not in e]
print (len(outside))
print (outside) 


29283 29284
VP -> XX PRT S S XX S
XX -> inch
0
[]
